Cleaning code from https://ryanwingate.com/projects/machine-learning-data-prep/adult/adult-cleaning/.

In [64]:
import pandas as pd
import numpy as np

In [65]:
adult_data_path = 'raw_adult/adult.data'
adult_test_path = 'raw_adult/adult.test'
cols = ['age','workclass','fnlwgt','education','education-num','marital-status',
        'occupation','relationship','race','sex','capital-gain', 'capital-loss',
        'hours-per-week', 'native-country','compensation']
a = (pd.read_csv(adult_data_path,
                 names=cols,
                 sep=', ',
                 engine='python')
     .append(pd.read_csv(adult_test_path,
                         skiprows=1,
                         names=cols,
                         sep=', ',
                         engine='python')))
print(a.shape)
a.head()

(48842, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,compensation
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [66]:
a = a.replace({'<=50K.' : '<=50K',
               '>50K.'  : '>50K'})

# Drop entries where workclass and occupation are unknown, and where workclass is Without-pay.
a = (a[(a['workclass']!='?')&
       (a['occupation']!='?')&
       (a['workclass']!='Without-pay')]
     .reset_index(drop=True))
a['idx'] = a.index

# Map the very small Armed-Forces category of occupation to Protective-serv.
a.loc[a['occupation']=='Armed-Forces','occupation'] = 'Protective-serv'

# Map Ages, Education, Workclass, and Weekly-Hours to smaller category set.
a.loc[a['workclass'].isin(['State-gov', 'Federal-gov', 'Local-gov']), 
      'employment-type'] = 'Government'
a.loc[a['workclass'].isin(['Self-emp-not-inc', 'Self-emp-inc']),      
      'employment-type'] = 'Self-Employed'
a.loc[a['workclass'].isin(['Private']),                               
      'employment-type'] = 'Privately-Employed'

a.loc[a['education-num'] <= 8,          'education'] = 'Less than High School'
a.loc[a['education-num'].isin([ 9,10]), 'education'] = 'High School'
a.loc[a['education-num'].isin([11,12]), 'education'] = 'Associates'
a.loc[a['education-num'].isin([13]),    'education'] = 'Bachelors'
a.loc[a['education-num'].isin([14]),    'education'] = 'Masters'
a.loc[a['education-num'].isin([15,16]), 'education'] = 'PhD/Professional'

a = a.rename(columns={'hours-per-week':'weekly-hours'})
a.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,weekly-hours,native-country,compensation,idx,employment-type
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0,Government
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1,Self-Employed
2,38,Private,215646,High School,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,2,Privately-Employed
3,53,Private,234721,Less than High School,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,3,Privately-Employed
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,4,Privately-Employed


In [67]:
# Convert all Categorical data to Numeric data
a['age raw'] = a['age']
# a['employment-type raw'] = a['employment-type']
a['education raw'] = a['education']
# a['occupation raw'] = a['occupation']
# a['race raw'] = a['race']
a['sex raw'] = a['sex']
a['weekly-hours raw'] = a['weekly-hours']
a['compensation raw'] = a['compensation']

# a['employment-type'] = a['employment-type'].astype('category')
# a['employment-type'] = a['employment-type'].cat.codes
a['education'] = a['education'].astype('category')
a['education'] = a['education'].cat.codes
# a['occupation'] = a['occupation'].astype('category')
# a['occupation'] = a['occupation'].cat.codes
# a['race'] = a['race'].astype('category')
# a['race'] = a['race'].cat.codes
a['sex'] = a['sex'].astype('category')
a['sex'] = a['sex'].cat.codes
a['compensation'] = a['compensation'].astype('category')
a['compensation'] = a['compensation'].cat.codes
                                                 
# print(a[['employment-type','employment-type raw','idx']].groupby(['employment-type', 'employment-type raw']).count())
# employment_types = sorted(a['employment-type raw'].unique()); print(employment_types)
print(a[['education','education raw','idx']].groupby(['education', 'education raw']).count())
# print(a[['occupation','occupation raw','idx']].groupby(['occupation', 'occupation raw']).count())
# occupations = sorted(a['occupation raw'].unique()); print(occupations)
# print(a[['race','race raw','idx']].groupby(['race', 'race raw']).count())
# races = sorted(a['race raw'].unique()); print(races)
print(a[['sex','sex raw','idx']].groupby(['sex', 'sex raw']).count())
print(a[['compensation','compensation raw','idx']].groupby(['compensation', 'compensation raw']).count())

a = a[['idx',
       'age',
       'employment-type',
       'education',
       'occupation',
       'race',
       'sex',
       'weekly-hours',
       'compensation']].copy()
print(a.shape)
# a.head()


                                   idx
education education raw               
0         Associates              3505
1         Bachelors               7772
2         High School            24991
3         Less than High School   5768
4         Masters                 2590
5         PhD/Professional        1386
               idx
sex sex raw       
0   Female   14912
1   Male     31100
                                 idx
compensation compensation raw       
0            <=50K             34592
1            >50K              11420
(46012, 9)


In [68]:
# One-hot encoding of categorical variables.
print(a.shape)
a = pd.get_dummies(a, columns=['employment-type', 'occupation', 'race'], drop_first=False)
# Move compensation to the end.
_c = a.pop('compensation') # remove column b and store it in df1
a['compensation'] = _c

print(a.shape)
a.head()

(46012, 9)
(46012, 27)


,idx,age,education,sex,weekly-hours,employment-type_Government,employment-type_Privately-Employed,employment-type_Self-Employed,occupation_Adm-clerical,occupation_Craft-repair,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,compensation
0,0,39,1,1,40,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,50,1,1,13,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,38,2,1,40,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,53,3,1,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4,28,1,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [69]:
a.to_csv('adult.csv', index=False)

---
### Don't do this one.

In [5]:
# Normalize Continuous Data.
print('   Mean Age = ' + str(a['age'].mean()))
print('Std Dev Age = ' + str(a['age'].std()))
a['age'] = (a['age'] - a['age'].mean()) / a['age'].std()

print('   Mean Weekly-Hours = ' + str(a['weekly-hours'].mean()))
print('Std Dev Weekly-Hours = ' + str(a['weekly-hours'].std()))
a['weekly-hours'] = (a['weekly-hours'] - a['weekly-hours'].mean()) / a['weekly-hours'].std()
a.head()


   Mean Age = 38.55261670868469
Std Dev Age = 13.199319129893762
   Mean Weekly-Hours = 40.95159958271755
Std Dev Weekly-Hours = 12.00779742417576


,idx,age,employment-type,education,occupation,race,sex,weekly-hours,compensation
0,0,0.033894,0,1,0,4,Male,-0.079248,0
1,1,0.867271,2,1,2,4,Male,-2.327787,0
2,2,-0.041867,1,2,4,4,Male,-0.079248,0
3,3,1.094555,1,3,4,2,Male,-0.079248,0
4,4,-0.799482,1,1,8,2,Female,-0.079248,0
